<a href="https://colab.research.google.com/github/bngrossmann/food-sales-predictions/blob/main/food_sales_predition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Food Sales Prediction
-Benjamin N. Grossmann

This project is currently a work in progress.

---

In [177]:
import pandas as pd

filename = "/content/sales_predictions.csv"
df_sales = pd.read_csv(filename)
df_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [178]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


There are 8523 rows of data across 12 columns.

A comparison of the datatypes for each column to the expected information type (infered from the information displayed in the `.head()` above) appears to be consistent.

* alphanumeric codes → `object`
    * Item_Identifier, Outlet_Identifier
* qualitative categories → `object`
    * Item_Fat_Content, Item_Type, Outlet_Size, Outlet_Location, Outlet_Type
* quantitative measurement → `float64`
    * Item_Weight, Item_Visibiltiy, Item_MRP, Item_Outlet_Sales
* year → `int64`
    * Outlet_Establishment_Year


---
Because there are Item_Identifier and Outlet_Identifier values, I am inferring these should be unique for each item and each outlet.

In [179]:
df_sales.nunique()

Item_Identifier              1559
Item_Weight                   415
Item_Fat_Content                5
Item_Visibility              7880
Item_Type                      16
Item_MRP                     5938
Outlet_Identifier              10
Outlet_Establishment_Year       9
Outlet_Size                     3
Outlet_Location_Type            3
Outlet_Type                     4
Item_Outlet_Sales            3493
dtype: int64

There are 1559 unique Item_Identifier values and 10 unique Outlet_Identifier values. Since each item can potentially be sold at each outlet, there will be at most 15,590 unique item-outlet pairs. Since there are only 8523 rows of data, there is no obvious red flag of duplicated data rows.

However, there could still be duplicated data rows present. Because each row should be unique, an attempt to drop any duplicates should still be made. 

There may also be a problem with inconsistent category names. Checking each of the qualitative categories can be done with a `.unique()` or `.value_counts()` method. I find the alignment of the  `.value_counts()` method to be easier to read quickly.

In [193]:
df_sales["Item_Fat_Content"].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

In [194]:
df_sales["Item_Type"].value_counts()

Fruits and Vegetables    1232
Snack Foods              1200
Household                 910
Frozen Foods              856
Dairy                     682
Canned                    649
Baking Goods              648
Health and Hygiene        520
Soft Drinks               445
Meat                      425
Breads                    251
Hard Drinks               214
Others                    169
Starchy Foods             148
Breakfast                 110
Seafood                    64
Name: Item_Type, dtype: int64

In [195]:
df_sales["Outlet_Type"].value_counts()

Supermarket Type1    5577
Grocery Store        1083
Supermarket Type3     935
Supermarket Type2     928
Name: Outlet_Type, dtype: int64

Only the Item_Fat_Content column has inconsistent category names. They can be corrected with a replacement method and quickly checked.

In [197]:
df_sales.replace({"LF":"Low Fat","low fat":"Low Fat","reg":"Regular"}, inplace=True)
df_sales["Item_Fat_Content"].value_counts()

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64

Then drop any rows that might be duplicates.

In [198]:
df_sales.drop_duplicates(keep=False, inplace=True)
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 865.6+ KB


The number of rows (8523) has not changed, so no rows has been identified as a duplicate at this time.

---
There are some columns in the dataframe that are missing values; the Item_Weight and Outlet_Size counts are less then 8523. How many are actually missing can be quickly calculated.

In [199]:
for column in df_sales:
    if df_sales.shape[0] - df_sales[column].count() > 0:
        print(df_sales.shape[0] - df_sales[column].count() , "values are missing from '"+ column +"'" )

1463 values are missing from 'Item_Weight'
2410 values are missing from 'Outlet_Size'


Because there are only two columns with missing data, there are only three kinds of rows with missing data
* Missing only Item_Weight
* Missing only Outlet_Size
* Missing both Item_Weight and Outlet_Size

To address the missing Item_Weight values, the corresponding Item_Identifier values should be examined. Items with the same Item_Identifier should be produced with uniform physical characteristics. The Item_Weights is a physical characteristic, so will be constant for a given Item_Identifier.

A step-by-step examination of the process for filling in the missing Item_Weight follows:

1. Select only the Item_Identifier and Item_Weight columns. This is to allow quick inspection of the process without the other columns causing distraction.
2. Sort the data by primarily by Item_Identifier, secondarily by Item_Weight. The result should show that Item_Weight is constant for a given Item_Identifier. The secondary sort criteria will also place the missing values at the end of each Item_Identifier cluster
3. Fill the missing Item_Weights with from the known value into the following missing value location (forward fill because the missing values are last in a cluster).
4. Drop the duplicate rows.

In [200]:
df_sales.sort_values(["Item_Identifier","Item_Weight"]).loc[:,"Item_Identifier":"Item_Weight"].fillna(method='ffill').drop_duplicates()

,Item_Identifier,Item_Weight
118,DRA12,11.600
1148,DRA24,19.350
1876,DRA59,8.270
3465,DRB01,7.390
1431,DRB13,6.115
...,...,...
1224,NCZ30,6.590
922,NCZ41,19.850
411,NCZ42,10.500
1267,NCZ53,9.600


There are 1559 rows, which is the number of unique Item_Identifiers. This means each Item_Identifier has a unique Item_Weight. (If there were more than 1559 rows, then an item would have more than one weight. This would clearly indicated either the uniqueness assumption was wrong or the replacement processes for the missing weight had an error.)

The process can be repeated for the Outlet_Identifier and Outlet_Size columns, following the same line of reasoning about uniqueness. (The Outlet_Establishment_Year can be left in this dataframe subset because it should be a constant for any given Outlet_Identifier. If it is not, then there is probably something further wrong with the data.)

In [201]:
df_sales.sort_values(["Outlet_Identifier","Outlet_Size"]).loc[:,"Outlet_Identifier":"Outlet_Size"].fillna(method='ffill').drop_duplicates()

,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size
3,OUT010,1998,NaN
4,OUT013,1987,High
9,OUT017,2007,High
1,OUT018,2009,Medium
23,OUT019,1985,Small
7,OUT027,1985,Medium
19,OUT035,2004,Small
8,OUT045,2002,Small
11,OUT046,1997,Small
0,OUT049,1999,Medium


There are only 10 rows, as expected. However Outlet_Identifier has a missing Outlet_Size. It appears there was no size value to begin with. We can quickly check that.

In [202]:
df_sales.sort_values(["Outlet_Identifier","Outlet_Size"]).loc[:,"Outlet_Identifier":"Outlet_Size"].value_counts()

Outlet_Identifier  Outlet_Establishment_Year  Outlet_Size
OUT027             1985                       Medium         935
OUT013             1987                       High           932
OUT035             2004                       Small          930
OUT046             1997                       Small          930
OUT049             1999                       Medium         930
OUT018             2009                       Medium         928
OUT019             1985                       Small          528
dtype: int64

Sacrebleu! There are only seven unique Outlet_Identifiers with an Outlet_Size. So three outlets don't have sizes recorded. A reduced outlet dataframe can be studied a little further. (Fortunately there are only 19 unique outlets.)

In [203]:
df_sales.loc[:,"Outlet_Identifier":"Outlet_Type"].drop_duplicates().sort_values(["Outlet_Identifier"])

,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
3,OUT010,1998,NaN,Tier 3,Grocery Store
4,OUT013,1987,High,Tier 3,Supermarket Type1
9,OUT017,2007,NaN,Tier 2,Supermarket Type1
1,OUT018,2009,Medium,Tier 3,Supermarket Type2
23,OUT019,1985,Small,Tier 1,Grocery Store
7,OUT027,1985,Medium,Tier 3,Supermarket Type3
19,OUT035,2004,Small,Tier 2,Supermarket Type1
8,OUT045,2002,NaN,Tier 2,Supermarket Type1
11,OUT046,1997,Small,Tier 1,Supermarket Type1
0,OUT049,1999,Medium,Tier 1,Supermarket Type1


There is no clear relationship between Outlet_Size and any of the other properties. Any analysis that depends upon the size of the outlet must exclude these three outlets for the time being.

It is tempting to assume they are medium sized as an average over the size range. However, that implicitly assumes any affects size has is monotonic or porportional across the size range and errors might "average out" between being wrong about not being small or high outlet sizes. The problem is that medium size might really have an extremum effect (the most or the least effect). This could greatly skew the analysis if medium is the wrong choice.

So, for now, excluding the three outlets of unknow size seems like the best action. If after an analysis of the remaining seven it is determined that size has little to no effect, then the three outlets could be reincorporated into the analysis.

In [204]:
has_outlet_size =  ~pd.isna(df_sales["Outlet_Size"])
df_sales = df_sales[has_outlet_size]
df_sales

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
...,...,...,...,...,...,...,...,...,...,...,...,...
8517,FDF53,20.750,Regular,0.083607,Frozen Foods,178.8318,OUT046,1997,Small,Tier 1,Supermarket Type1,3608.6360
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


After removing the outlets of unknown size, the item weights can be filled in. A quick .info() check can be done to verify no values are missing.

In [205]:
df_filled = df_sales
df_filled = df_filled.sort_values(["Item_Identifier","Item_Weight"])
df_filled.loc[:,"Item_Identifier":"Item_Weight"] = df_filled.loc[:,"Item_Identifier":"Item_Weight"].fillna(method='ffill')
df_filled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6113 entries, 1245 to 7166
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            6113 non-null   object 
 1   Item_Weight                6113 non-null   float64
 2   Item_Fat_Content           6113 non-null   object 
 3   Item_Visibility            6113 non-null   float64
 4   Item_Type                  6113 non-null   object 
 5   Item_MRP                   6113 non-null   float64
 6   Outlet_Identifier          6113 non-null   object 
 7   Outlet_Establishment_Year  6113 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       6113 non-null   object 
 10  Outlet_Type                6113 non-null   object 
 11  Item_Outlet_Sales          6113 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 620.9+ KB


All columns have 6113 values down all 6113 rows.

---
The summary statistics can be shown for the remaing numerical values in each column.

In [206]:
df_filled.describe().round(2)

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,6113.00,6113.00,6113.00,6113.00,6113.00
mean,12.91,0.06,141.26,1995.79,2322.69
std,4.67,0.05,62.23,8.84,1741.59
min,4.56,0.00,31.29,1985.00,33.96
25%,8.78,0.03,94.01,1987.00,974.73
50%,12.65,0.05,143.18,1997.00,1928.16
75%,17.00,0.09,185.89,2004.00,3271.08
max,21.35,0.33,266.89,2009.00,13086.96


Note that finding statistics on the Outlet_Establishment_Year may seems odd. However, it might be important how an outlet's age might affect sales. However, the count statistic for the year doesn't have any clear meaning an is purely an artifact of the calender system. The other year statistics can be use to find the corresponding age statistics.